In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from tqdm import tqdm
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle
import tensorflow as tf
import math
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from sklearn.metrics import silhouette_samples
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering

In [2]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [3]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


In [4]:
dataset = []
data = np.array(df)
for i in df.index:
    l = eval(df["Playlists"][i])
#     if len(l) > 1:
#         continue
    data[i][-1] = l
    dataset.append(data[i])
dataset = np.array(dataset)
        

In [5]:
print(len(df))
print(len(dataset))

11159
11159


In [6]:
# counts = []
# for i in range(169):
#     counts.append(0)
# for i in range(len(dataset)):
#     counts[int(dataset[i][-1])] += 1
# c = 0
# playlists_to_keep = set()
# for i in range(len(counts)):
#     if counts[i] >= 120:
#         c += 1
#         playlists_to_keep.add(str(i))        
# print(c)

In [7]:
# temp_dataset = []
# for c in dataset:
#     if str(c[-1]) in playlists_to_keep:
#         temp_dataset.append(c)
# dataset = temp_dataset
# dataset = np.array(dataset)
        

In [8]:
X = dataset[:, 1:-1]
y = dataset[:, -1]

In [9]:
standardScalar = StandardScaler()
X = standardScalar.fit_transform(X)

In [10]:
# pca = PCA(n_components = 15)
# X = pca.fit_tranform(X)
# pca.explained_variance_

In [11]:
svd = TruncatedSVD(n_components = 15, random_state = 0)
X = svd.fit_transform(X)
# pca.explained_variance_

In [12]:
print(X.shape)
print(y.shape)
print(X[0])

(11159, 15)
(11159,)
[-1.81088185 -1.63204109 -0.73234896 -0.01666838 -0.70166087  1.39663556
 -1.31109854  0.13860771 -0.70312456 -0.27157586 -0.19136596 -0.16054815
 -1.19500578  0.07239146 -0.22634698]


In [ ]:
for i in range(8, 9):
    kmeans = KMeans(n_clusters=i, random_state=10).fit(X)
    print(i, silhouette_score(X, kmeans.labels_, metric='euclidean')) 
    pred = kmeans.fit_predict(X)
    
    xx=TSNE(n_components = 2 )
    X_TSNE=xx.fit_transform(X)
    tsne_df = pd.DataFrame(columns = ['tsvd1','tsvd2','Ytrain'])
    tsne_df["tsvd1"]=X_TSNE[:,0]
    tsne_df["tsvd2"]=X_TSNE[:,1]
    tsne_df["Ytrain"]=pred
    # creating a dataframe of both dimensions of TSNE model and Ytrain to visualize plots

    # plotting our TSNE plot with labels on side
    plt.figure(figsize=(16,10))
    sns.scatterplot(x="tsvd1", y="tsvd2", hue="Ytrain",data=tsne_df,palette=sns.color_palette("hls", i),legend="full")
    plt.show()
    

In [13]:
def compareClustersWithOriginalPlaylists(pred, cluster_centres):
    num_clusters = len(cluster_centres)
    metrics_clusters = []
    for i in range(num_clusters):
        s = 0
        c = 0 
        for j in range(len(X)):
            if pred[j] == i:
                s1 = 0
                for k in range(15):
                    s1 += (X[j][k] - cluster_centres[i][k])**2
                s1 = s1**(0.5)
                s += s1
                c += 1
        s = s/c
        metrics_clusters.append(s)
        
    print(sum(metrics_clusters)/num_clusters)
    
    
    playlist_centres = []
    playlist_counts = []
    for i in range(169):
        a = []
        for j in range(15):
            a.append(0)
        playlist_centres.append(a)
        playlist_counts.append(0)
    
    for i in range(len(X)):
        for c in y[i]:
            playlist_counts[c] += 1
            for j in range(15):
                playlist_centres[c][j] += X[i][j]
#     print(playlist_counts)
    for i in range(169):
        for j in range(15):
            playlist_centres[i][j] = playlist_centres[i][j]/playlist_counts[i]

    
    metrics_playlist = []
    for i in range(169):
        s = 0
        c = 0 
        for j in range(len(X)):
            if i in y[j]:
                s1 = 0
                for k in range(15):
                    s1 += (X[j][k] - playlist_centres[i][k])**2
                s1 = s1**(0.5)
                s += s1
                c += 1
        s = s/c
        metrics_playlist.append(s)
    
    print(sum(metrics_playlist)/169)

        
    
    
    
    
    


    
    

In [14]:
def computeCentres(X, pred, num_clusters):
    cluster_centres = []
    cluster_counts = []
    for i in range(num_clusters):
        a = []
        for j in range(15):
            a.append(0)
        cluster_centres.append(a)
        cluster_counts.append(0)
    
    for i in range(len(X)):
        c = pred[i]
        if c == -1:
#             dbscan noise point
            continue
        cluster_counts[c] += 1
        for j in range(15):
            cluster_centres[c][j] += X[i][j]
    
    for i in range(num_clusters):
        for j in range(15):
            cluster_centres[i][j] = cluster_centres[i][j]/cluster_counts[i]    
    return cluster_centres

Kmeans

In [15]:
kmeans = KMeans(n_clusters=8, random_state=69).fit(X)
pred = kmeans.predict(X)

In [16]:
cluster_centres = computeCentres(X, pred, 8)

In [17]:
compareClustersWithOriginalPlaylists(pred, cluster_centres)

3.446960645600497
3.468469618813997


In [18]:
silhouette_score(X, pred)

0.1295546610545091

In [ ]:
list1 = [3, 5, 7, 8, 9, 10, 11, 13, 15, 18, 20, 23, 26, 30]
xvals = []
yvals = []
for val in tqdm(list1):
    kmeans = KMeans(n_clusters=val, random_state=10)
    kmeans.fit(X)
    pred = kmeans.predict(X)
    xvals.append(val)
    ss = silhouette_score(X, pred)
    yvals.append(ss)
    print(val, ss)


In [ ]:
plt.figure(figsize = (10, 6))
plt.plot(xvals, yvals)
plt.xlabel('n_clusters', fontsize = 14)
plt.ylabel('Silhouette Score', fontsize = 14)
plt.show()

In [ ]:
print(xvals)
print(yvals)

In [ ]:
f = open("clustering_models/kmeans.pkl", "wb")
pickle.dump(kmeans, f)
f.close()

DBSCAN

In [19]:
pred = DBSCAN(eps = 2, min_samples = 40).fit_predict(X)

In [ ]:
pred

In [ ]:
s = set()
for c in pred:
    s.add(c)
counts = {}
for c in s:
    counts[c] = 0
for c in pred:
    counts[c] += 1
print(len(s))
print(counts)

In [ ]:
cluster_centres = computeCentres(X, pred, len(s)-1)

In [ ]:
len(cluster_centres)

In [ ]:
compareClustersWithOriginalPlaylists(pred, cluster_centres)

In [ ]:
print(len(X))
print(len(pred))

In [ ]:
print(pred[0])

In [ ]:
print(cluster_centres.shape)

Agglomerative Clustering

In [20]:
agglo = AgglomerativeClustering(n_clusters = 8).fit(X)
pred = agglo.fit_predict(X)

In [21]:
print(pred)

[0 7 0 ... 0 0 0]


In [22]:
s = set()
for c in pred:
    s.add(c)
counts = {}
for c in s:
    counts[c] = 0
for c in pred:
    counts[c] += 1
print(len(s))
print(counts)

8
{0: 5762, 1: 2396, 2: 1014, 3: 157, 4: 655, 5: 68, 6: 21, 7: 1086}


In [23]:
cluster_centres = computeCentres(X, pred, 8)

In [24]:
compareClustersWithOriginalPlaylists(pred, cluster_centres)

3.9743668067623235
3.468469618813997


In [25]:
silhouette_score(X, pred)

0.11749039550867599

In [ ]:
list1 = [3, 5, 7, 8, 9, 10, 11, 13, 15, 18, 20, 23, 26, 30]
xvals = []
yvals = []
for val in tqdm(list1):
    agglo = AgglomerativeClustering(n_clusters=val)
    pred = agglo.fit_predict(X)
    xvals.append(val)
    ss = silhouette_score(X, pred)
    yvals.append(ss)
    print(val, ss)


In [26]:
f = open("clustering_models/agglo.pkl", "wb")
pickle.dump(agglo, f)
f.close()

Spectral Clustering

In [ ]:
spectral = SpectralClustering(n_clusters=8).fit(X)
pred = spectral.fit_predict(X)

In [ ]:
print(pred)

In [ ]:
# for n_clusters in range(2,11):
#     # Create a subplot with 1 row and 2 columns
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     fig.set_size_inches(18, 7)

#     # The 1st subplot is the silhouette plot
#     # The silhouette coefficient can range from -1, 1 but in this example all
#     # lie within [-0.1, 1]
#     ax1.set_xlim([-0.1, 1])
#     # The (n_clusters+1)*10 is for inserting blank space between silhouette
#     # plots of individual clusters, to demarcate them clearly.
#     ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

#     # Initialize the clusterer with n_clusters value and a random generator
#     # seed of 10 for reproducibility.
#     clusterer = KMeans(n_clusters=n_clusters, random_state=10)
#     cluster_labels = clusterer.fit_predict(X)

#     # The silhouette_score gives the average value for all the samples.
#     # This gives a perspective into the density and separation of the formed
#     # clusters
#     silhouette_avg = silhouette_score(X, cluster_labels)
#     print("For n_clusters =", n_clusters,
#           "The average silhouette_score is :", silhouette_avg)

#     # Compute the silhouette scores for each sample
#     sample_silhouette_values = silhouette_samples(X, cluster_labels)

#     y_lower = 10
#     for i in range(n_clusters):
#         # Aggregate the silhouette scores for samples belonging to
#         # cluster i, and sort them
#         ith_cluster_silhouette_values = \
#             sample_silhouette_values[cluster_labels == i]

#         ith_cluster_silhouette_values.sort()

#         size_cluster_i = ith_cluster_silhouette_values.shape[0]
#         y_upper = y_lower + size_cluster_i

#         color = cm.nipy_spectral(float(i) / n_clusters)
#         ax1.fill_betweenx(np.arange(y_lower, y_upper),
#                           0, ith_cluster_silhouette_values,
#                           facecolor=color, edgecolor=color, alpha=0.7)

#         # Label the silhouette plots with their cluster numbers at the middle
#         ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

#         # Compute the new y_lower for next plot
#         y_lower = y_upper + 10  # 10 for the 0 samples

#     ax1.set_title("The silhouette plot for the various clusters.")
#     ax1.set_xlabel("The silhouette coefficient values")
#     ax1.set_ylabel("Cluster label")

#     # The vertical line for average silhouette score of all the values
#     ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

#     ax1.set_yticks([])  # Clear the yaxis labels / ticks
#     ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

#     # 2nd Plot showing the actual clusters formed
#     colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
#     ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
#                 c=colors, edgecolor='k')

#     # Labeling the clusters
#     centers = clusterer.cluster_centers_
#     # Draw white circles at cluster centers
#     ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
#                 c="white", alpha=1, s=200, edgecolor='k')

#     for i, c in enumerate(centers):
#         ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
#                     s=50, edgecolor='k')

#     ax2.set_title("The visualization of the clustered data.")
#     ax2.set_xlabel("Feature space for the 1st feature")
#     ax2.set_ylabel("Feature space for the 2nd feature")

#     plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
#                   "with n_clusters = %d" % n_clusters),
#                  fontsize=14, fontweight='bold')

# plt.show()